# Advent of Code 2019
## [Day 7: Amplification Circuit](https://adventofcode.com/2019/day/7)

In [ ]:
from pathlib import Path
import sys
if str(Path('../..').resolve()) not in sys.path:
    sys.path.insert(0, str(Path('../..').resolve()))

from intcode import Intcode, Operation, ParameterMode

In [2]:
import numpy as np
from collections import deque

In [3]:
with open("input.txt") as f:
    input_program = [*map(int, f.read().split(","))]
input_program[:5]

[3, 8, 1001, 8, 10]

In [4]:
Intcode(input_program)

<Intcode pc:0/519 in:0 out:[]>

In [5]:
Intcode.__name__

'Intcode'

In [6]:
m = Intcode(input_program)
m

<Intcode pc:0/519 in:0 out:[]>

In [7]:
print(m)

>   0: Input                *8
    2: Add                  *8   10   *8
    6: JumpIfTrue            1   *0
    9: 0
   10: 21
   11: 42
   12: 63
   13: 76
   14: Add                 114 *195 *276
   18: 57
   19: 38
   20: Halt              
   21: Input                *9
   23: Add                   2   *9   *9
   27: Multiply              5   *9   *9
   31: Add                  *9    3   *9
   35: Multiply             *9    5   *9
   39: Output               *9
   41: Halt              
   42: Input                *9
   44: Add                   4   *9   *9
   48: Multiply              5   *9   *9
   52: Add                  *9    5   *9
   56: Multiply              2   *9   *9
   60: Output               *9
   62: Halt              
   63: Input                *9
   65: Add                  *9    3   *9
   69: Multiply             *9    5   *9
   73: Output               *9
   75: Halt              
   76: Input                *9
   78: Multiply             *9    2   *9
   82: Add

In [8]:
o = Operation(2)

In [11]:
m.memory[:5]

[3, 8, 1001, 8, 10]

---

In [12]:
for phase_setting in [0, 1, 2, 3, 4]:
    for input_signal in range(1000):
        

IndexError: pop from an empty deque